In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

from time import sleep, time
from threading import Thread
from scipy.cluster.hierarchy import fcluster
from scipy.stats import zscore

from testing import TestFactory, ClusteredInfo
from dtw import dtw as cur_dtw
from dtw_wrapper import DtwWrapper

from IPython.display import Markdown
from tqdm import tqdm, tqdm_notebook

In [2]:
CLUSTER_SIZES = [4, 12, 24, 36, 48]
N_CLUST = 30

def metric_Q(info):
    Q = 0
    norm_koeff = 0
    for i in range(info.count): 
        for j in range(info.count):
            if info.label[i] == info.label[j]:
                norm_koeff += 1
            if info.label[i] == info.label[j] and info.clusters_labels[i] == info.clusters_labels[j]:
                Q += 1
                
    return Q / norm_koeff


def metric_cluster(info):
    Q = 0
    for i in range(1, info.cluster_num + 1):
        ind, counts = np.unique(info.label[np.where(info.clusters_labels == i)[0]], return_counts=True)
        Q += counts.max() / counts.sum()

    return Q / info.cluster_num

def metric_cluster_vlad(info):
    Q = 0
    for i in range(1, info.cluster_num + 1):
        ind, counts = np.unique(info.label[np.where(info.clusters_labels == i)[0]], return_counts=True)
        Q += (counts.max()) ** 2 / len(np.where(info.clusters_labels == i)[0]) / counts.sum() 

    return Q / info.cluster_num

def norm_1(x, y):
    return np.linalg.norm(x - y, ord=1)
def norm_2(x, y):
    return np.linalg.norm(x - y, ord=2)

def pipeline(info, autoregression=False, show_results=True):
    metrics = {0: [], 1: []}
    
    info.cluster(N_CLUST)
    Q1 = metric_cluster(info)
    Q2 = metric_cluster_vlad(info)
        
    for cluster_size in CLUSTER_SIZES:
        info.cluster(cluster_size)
        Q1 = metric_cluster(info)
        Q2 = metric_cluster_vlad(info)
        metrics[0].append(Q1)
        metrics[1].append(Q2)
        if show_results:
            print("{0:4}: Q1:{1:.4f} | Q2:{2:.4f}".format(cluster_size, Q1, Q2))
    
    if not show_results:
        return metrics[0], metrics[1]

    index = info.stats.head(10).index.values
    classifier_stat = {}
    for i in info.stats.index:
        classifier_stat[i] = pd.Series(info.label[np.where(info.clusters_labels == i)[0]]).value_counts()

    display(pd.DataFrame(classifier_stat).fillna(0).iloc[:, :30])
    
    for i in index[:6]:
        info.clusters_compare_table(label=i, z_normalize=True)
    plt.show()
    
    display(Markdown("#### До выравнивания"))
    for i in index[:6]:
        info.comparing_at_one(i, num_series=10, z_normalize=True)
    plt.show()
    
    if not autoregression:
        display(Markdown("#### С выравниванием"))
        for i in index[:6]:
            info.allignment_to_random(i, num_series=10, z_normalize=True)
        plt.show()
        
    return metrics[0], metrics[1]
        
def repeat_test(args, kwargs, n_repeat, sample_size, autoregression=False):
    metrics = []
    for i in range(n_repeat):
        tests = TestFactory(random_state=i)
        x = tests.set_sample(sample_size)
        info = tests.ar_clustering() if autoregression else tests.test_dtw(*args, **kwargs)
        metrics.append(pipeline(info, autoregression, show_results=False))

    print("\n---- Metrics ----\n")
    print("  ".join(["{0:0.3f}+-{1:0.3f}".format(s, m) for s, m in zip(
        np.mean(np.array(metrics)[:, 0, :], 0),
        np.std(np.array(metrics)[:, 0, :], 0))]))
    print("  ".join(["{0:0.3f}+-{1:0.3f}".format(s, m) for s, m in zip(
            np.mean(np.array(metrics)[:, 1, :], 0),
            np.std(np.array(metrics)[:, 1, :], 0))]))

    print("{0:0.4f} +- {1:0.4f}".format(np.mean(np.array(metrics)[:, 0, :]), np.std(np.array(metrics)[:, 0, :])))
    print("{0:0.4f} +- {1:0.4f}".format(np.mean(np.array(metrics)[:, 1, :]), np.std(np.array(metrics)[:, 1, :])))

    return metrics

# DTW
Стоит играться с функциями расстояния между кластерами.  
Сейчас стоит `complete`: $$d(X, Y) = max(dist(x, y))$$

Более менее работает с `weighted` и `average`.  
[Подробнее](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.linkage.html#scipy.cluster.hierarchy.linkage)

Практически во всех способах, остается первый кластер, размер которого самый большой. В нем, зачастую, все перемемашно.

In [ ]:
for i in range(3):
    tests = TestFactory(random_state=i)
    x = tests.set_sample(2000)
    info = tests.test_dtw(cur_dtw, norm_1, 
                      dtw_args={"z_normalize": True, "l": 0.2},
                      cluster_dist="complete")

    tests.dtw_wrapper.dump()
    info = tests.test_dtw(cur_dtw, norm_2, 
                      dtw_args={"z_normalize": True, "l": 0.2},
                      cluster_dist="complete")

    tests.dtw_wrapper.dump()

## L_1

### complete

In [3]:
metrics = repeat_test([cur_dtw, norm_1],
                      {"dtw_args": {"z_normalize": True, "l": 0.2}, "cluster_dist": "complete"},
                     20, 600)

Elapsed time: 0.480
Elapsed time: 0.474
Elapsed time: 0.485
Elapsed time: 0.472
Elapsed time: 0.509
Elapsed time: 0.579
Elapsed time: 0.525
Elapsed time: 0.493
Elapsed time: 0.585
Elapsed time: 0.464
Elapsed time: 0.484
Elapsed time: 0.480
Elapsed time: 0.608
Elapsed time: 0.473
Elapsed time: 0.464
Elapsed time: 0.503
Elapsed time: 0.489
Elapsed time: 0.498
Elapsed time: 0.480
Elapsed time: 0.523

---- Metrics ----

0.421+-0.075  0.462+-0.038  0.502+-0.028  0.546+-0.023  0.580+-0.022
0.218+-0.089  0.243+-0.046  0.278+-0.034  0.328+-0.028  0.370+-0.028
0.5019 +- 0.0709
0.2874 +- 0.0750


In [4]:
tests = TestFactory(random_state=2)
_ = tests.set_sample(600)
info = tests.test_dtw(cur_dtw, norm_1, 
                      dtw_args={"z_normalize": True, "l": 0.2},
                      cluster_dist="complete")

# pipeline(info)

Elapsed time: 0.634


### weighted

In [5]:
metrics = repeat_test(
    [cur_dtw, norm_1],
    {"dtw_args": {"z_normalize": True, "l": 0.2}, "cluster_dist": "weighted"},
    20, 600)
info = tests.test_dtw(cur_dtw, norm_1, 
                      dtw_args={"z_normalize": True, "l": 0.2},
                      cluster_dist="weighted")
# pipeline(info)

Elapsed time: 0.454
Elapsed time: 0.447
Elapsed time: 0.473
Elapsed time: 0.544
Elapsed time: 0.459
Elapsed time: 0.484
Elapsed time: 0.466
Elapsed time: 0.477
Elapsed time: 0.599
Elapsed time: 0.443
Elapsed time: 0.448
Elapsed time: 0.495
Elapsed time: 0.539
Elapsed time: 0.597
Elapsed time: 0.517
Elapsed time: 0.477
Elapsed time: 0.445
Elapsed time: 0.459
Elapsed time: 0.558
Elapsed time: 0.533

---- Metrics ----

0.507+-0.072  0.544+-0.038  0.569+-0.031  0.601+-0.023  0.629+-0.021
0.312+-0.104  0.332+-0.054  0.357+-0.040  0.397+-0.030  0.434+-0.029
0.5703 +- 0.0594
0.3664 +- 0.0730
Elapsed time: 0.520


### average

In [6]:
metrics = repeat_test(
    [cur_dtw, norm_1],
    {"dtw_args": {"z_normalize": True, "l": 0.2}, "cluster_dist": "average"},
    20, 600)
info = tests.test_dtw(cur_dtw, norm_1, 
                      dtw_args={"z_normalize": True, "l": 0.2},
                      cluster_dist="average")
# pipeline(info)

Elapsed time: 0.571
Elapsed time: 0.469
Elapsed time: 0.455
Elapsed time: 0.449
Elapsed time: 0.480
Elapsed time: 0.454
Elapsed time: 0.463
Elapsed time: 0.441
Elapsed time: 0.458
Elapsed time: 0.439
Elapsed time: 0.465
Elapsed time: 0.563
Elapsed time: 0.449
Elapsed time: 0.493
Elapsed time: 0.470
Elapsed time: 0.445
Elapsed time: 0.449
Elapsed time: 0.453
Elapsed time: 0.600
Elapsed time: 0.610

---- Metrics ----

0.545+-0.064  0.566+-0.039  0.587+-0.030  0.613+-0.033  0.636+-0.025
0.363+-0.089  0.358+-0.053  0.377+-0.039  0.412+-0.046  0.443+-0.035
0.5893 +- 0.0520
0.3906 +- 0.0647
Elapsed time: 0.566


## L_2

### complete

In [7]:
metrics = repeat_test(
    [cur_dtw, norm_2],
    {"dtw_args": {"z_normalize": True, "l": 0.2}, "cluster_dist": "complete"},
    20, 600)
info = tests.test_dtw(cur_dtw, norm_2, 
                      dtw_args={"z_normalize": True, "l": 0.2},
                      cluster_dist="complete")

tests.dtw_wrapper.dump()
# pipeline(info)

Elapsed time: 0.434
Elapsed time: 0.450
Elapsed time: 0.439
Elapsed time: 0.451
Elapsed time: 0.484
Elapsed time: 0.451
Elapsed time: 0.457
Elapsed time: 0.479
Elapsed time: 0.447
Elapsed time: 0.649
Elapsed time: 0.675
Elapsed time: 0.915
Elapsed time: 0.489
Elapsed time: 0.476
Elapsed time: 0.516
Elapsed time: 0.492
Elapsed time: 0.509
Elapsed time: 0.522
Elapsed time: 0.549
Elapsed time: 0.775

---- Metrics ----

0.421+-0.062  0.464+-0.037  0.504+-0.027  0.543+-0.025  0.581+-0.023
0.222+-0.072  0.250+-0.045  0.283+-0.033  0.326+-0.031  0.372+-0.029
0.5028 +- 0.0678
0.2905 +- 0.0700
Elapsed time: 0.469


### weighted

In [8]:
metrics = repeat_test(
    [cur_dtw, norm_2],
    {"dtw_args": {"z_normalize": True, "l": 0.2}, "cluster_dist": "weighted"},
    20, 600)
info = tests.test_dtw(cur_dtw, norm_1, 
                      dtw_args={"z_normalize": True, "l": 0.2},
                      cluster_dist="weighted")
# pipeline(info)

Elapsed time: 0.505
Elapsed time: 0.441
Elapsed time: 0.441
Elapsed time: 0.500
Elapsed time: 0.483
Elapsed time: 0.440
Elapsed time: 0.496
Elapsed time: 0.453
Elapsed time: 0.461
Elapsed time: 0.575
Elapsed time: 0.545
Elapsed time: 0.449
Elapsed time: 0.924
Elapsed time: 0.549
Elapsed time: 0.532
Elapsed time: 0.465
Elapsed time: 0.458
Elapsed time: 0.488
Elapsed time: 0.578
Elapsed time: 0.440

---- Metrics ----

0.469+-0.072  0.531+-0.038  0.566+-0.031  0.596+-0.028  0.626+-0.030
0.264+-0.091  0.316+-0.051  0.353+-0.040  0.391+-0.038  0.431+-0.038
0.5576 +- 0.0694
0.3509 +- 0.0804
Elapsed time: 0.461


### average

In [9]:
metrics = repeat_test(
    [cur_dtw, norm_2],
    {"dtw_args": {"z_normalize": True, "l": 0.2}, "cluster_dist": "average"},
    20, 600)
info = tests.test_dtw(cur_dtw, norm_1, 
                      dtw_args={"z_normalize": True, "l": 0.2},
                      cluster_dist="average")
# pipeline(info)

Elapsed time: 0.444
Elapsed time: 0.460
Elapsed time: 0.594
Elapsed time: 0.613
Elapsed time: 0.614
Elapsed time: 0.465
Elapsed time: 0.504
Elapsed time: 0.567
Elapsed time: 0.460
Elapsed time: 0.429
Elapsed time: 0.478
Elapsed time: 0.511
Elapsed time: 0.483
Elapsed time: 0.486
Elapsed time: 0.446
Elapsed time: 0.467
Elapsed time: 0.435
Elapsed time: 0.454
Elapsed time: 0.493
Elapsed time: 0.472

---- Metrics ----

0.544+-0.064  0.588+-0.041  0.593+-0.026  0.617+-0.025  0.642+-0.026
0.353+-0.092  0.386+-0.053  0.392+-0.033  0.421+-0.033  0.455+-0.036
0.5969 +- 0.0511
0.4013 +- 0.0642
Elapsed time: 0.484


## Autoregression

In [10]:
metrics = repeat_test(None, None, 20, 600, True)
tests = TestFactory(random_state=1)
_ = tests.set_sample(600)
info = tests.ar_clustering()
# pipeline(info, autoregression=True)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 600/600 [00:40<00:00, 14.74it/s]


Elapsed time: 40.739


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 600/600 [00:36<00:00, 16.31it/s]


Elapsed time: 36.616


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 600/600 [00:40<00:00, 16.39it/s]


...

Elapsed time: 29.117


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 600/600 [00:28<00:00, 21.10it/s]


Elapsed time: 28.599

---- Metrics ----

0.529+-0.047  0.619+-0.032  0.660+-0.037  0.695+-0.028  0.718+-0.026
0.325+-0.062  0.438+-0.044  0.494+-0.054  0.544+-0.040  0.575+-0.035
0.6444 +- 0.0753
0.4751 +- 0.1003


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 600/600 [00:28<00:00, 20.77it/s]


Elapsed time: 28.905
